<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [75]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [76]:
# a)
df = pd.DataFrame()
df_anio1 = pd.read_csv(archivos_anio[0], sep=',')
df_anio2 = pd.read_csv(archivos_anio[1], sep=',')
df_anio1.columns = df_anio1.columns.str.lower()
df_anio2.columns = df_anio2.columns.str.lower()
df_anio = pd.concat([df_anio1, df_anio2])
df_anio.head()

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN


In [77]:
#b)
duplicados=df_codigos[df_codigos.duplicated("codigo_iso")]
duplicados

df_codigos=df_codigos.drop_duplicates(subset=['codigo_iso'])
df_codigos

,codigo_iso,pais
0,AFG,Afghanistán
1,AGO,Angola
2,ALB,Albania
3,AND,Andorra
4,ARE,Emiratos Árabes Unidos
...,...,...
175,WSM,Samoa
176,YEM,Yemen
177,ZAF,Sudáfrica
178,ZMB,Zambia


In [78]:
# c)

df = pd.merge(df_anio, df_codigos, on='codigo_iso')
df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [79]:
# Estructura Dataframe
print("cantidad de filas", df.shape[0])
print("cantidad de columnas", df.shape[1])
print("nombres de las columnas", df.columns)
print("tipo de datos", df.dtypes)
print("todas las columnas tienen los tipos de datos esperados")

cantidad de filas 3060
cantidad de columnas 5
nombres de las columnas Index(['codigo_iso', 'anio', 'indice', 'ranking', 'pais'], dtype='object')
tipo de datos codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object
todas las columnas tienen los tipos de datos esperados


In [80]:
#Resumen Estadistico
df.describe()


,anio,indice,ranking
count,3060.000000,2664.000000,2837.000000
mean,2009.941176,205.782316,477.930913
std,5.786024,2695.525264,6474.935347
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.295000,34.000000
50%,2009.000000,28.000000,70.000000
75%,2015.000000,41.227500,110.000000
max,2019.000000,64536.000000,121056.000000


En indice y ranking el promedio es más pequeño que la media, los valores observados son muy altos con respecto al $75\%$ de los datos, además de una alta desviación estandar, mostrando una alta dispersion en los datos de ambas columnas.

Para indice:
Minimo: $0.0$ , Máximo: $64536.0$ , Promedio: $205.78$





In [81]:
valor_max_indice=df["indice"].max()
valor_min_indice=df["indice"].min()
print("Los paises con mayor indice son:", df[df["indice"]==valor_max_indice]["pais"].unique())
print("Los paises con menor indice son:", df[df["indice"]==valor_min_indice]["pais"].unique())

valor_max_ranking=df["ranking"].max()
valor_min_ranking=df["ranking"].min()
print("Los paises con mayor ranking son:", df[df["ranking"]==valor_max_ranking]["pais"].unique())
print("Los paises con menor ranking son:", df[df["ranking"]==valor_min_ranking]["pais"].unique())

Los paises con mayor indice son: ['Kosovo']
Los paises con menor indice son: ['Dinamarca' 'Finlandia' 'Irlanda' 'Noruega' 'Suecia' 'Suiza' 'Islandia'
 'Países Bajos']
Los paises con mayor ranking son: ['Kosovo']
Los paises con menor ranking son: ['Finlandia' 'Islandia' 'Países Bajos' 'Noruega' 'Suiza' 'Dinamarca'
 'Irlanda' 'Eslovaquia' 'Luxemburgo' 'Estonia' 'Austria' 'Nueva Zelanda']


In [82]:
#Datos Faltantes

df.isnull().sum()

,0
codigo_iso,0
anio,0
indice,396
ranking,223
pais,0


Hay 396 datos faltantes en Indice y 223 en Ranking



In [83]:
df_anio.isnull().mean()

,0
codigo_iso,0.000000
anio,0.000000
indice,0.129412
ranking,0.072876


El porcentaje para Indice es cercano a $13\%$ y para Ranking es cercano a $7\%$

Podemos observar en la tabla que no hay ninguna columna sobre el $30\%$

In [84]:
#Unicidad y duplicados

print("Cantidad de paises distintos:", df["pais"].nunique())
print("Cantidad de años distintos:", df["anio"].nunique())
print("Cantidad de duplicados:", df.duplicated().sum())

Cantidad de paises distintos: 179
Cantidad de años distintos: 17
Cantidad de duplicados: 0


In [85]:
#Validacion cruzada de columnas

inconsistent_codes = df.groupby('codigo_iso')['pais'].nunique()
inconsistent_codes = inconsistent_codes[inconsistent_codes > 1]
inconsistent_codes


,pais
codigo_iso,


No hay inconsistencias




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [88]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df[df["codigo_iso"].isin(america)]

#a)
for anio in df_america["anio"].unique():
  df_anio_filtro= df_america[df_america["anio"] == anio]
  if not df_anio_filtro.empty:
    df_anio_filtro_limpio = df_anio_filtro.dropna(subset=['indice'])
    if not df_anio_filtro_limpio.empty:
      pais_min_indice_idx = df_anio_filtro_limpio["indice"].idxmin()
      pais_max_indice_idx = df_anio_filtro_limpio["indice"].idxmax()
      print("En el año", anio, ":")
      print("  País con menor índice (mayor libertad): {df_america.loc[pais_min_indice_idx, 'pais']}")
      print("  País con mayor índice (menor libertad): {df_america.loc[pais_max_indice_idx, 'pais']}")
      print()

#b)
df_america_grouped = df_america.groupby('anio')
min_indice_idx = df_america_grouped['indice'].idxmin().dropna()
max_indice_idx = df_america_grouped['indice'].idxmax().dropna()

print("Paises con menor indice (mayor libertad) en cada año:")
display(df_america.loc[min_indice_idx, ['anio', 'pais']])


print("\nPaises con mayor indice (menor libertad) en cada año:")
display(df_america.loc[max_indice_idx, ['anio', 'pais']])


En el año 2001 :
  País con menor índice (mayor libertad): {df_america.loc[pais_min_indice_idx, 'pais']}
  País con mayor índice (menor libertad): {df_america.loc[pais_max_indice_idx, 'pais']}

En el año 2002 :
  País con menor índice (mayor libertad): {df_america.loc[pais_min_indice_idx, 'pais']}
  País con mayor índice (menor libertad): {df_america.loc[pais_max_indice_idx, 'pais']}

En el año 2003 :
  País con menor índice (mayor libertad): {df_america.loc[pais_min_indice_idx, 'pais']}
  País con mayor índice (menor libertad): {df_america.loc[pais_max_indice_idx, 'pais']}

En el año 2004 :
  País con menor índice (mayor libertad): {df_america.loc[pais_min_indice_idx, 'pais']}
  País con mayor índice (menor libertad): {df_america.loc[pais_max_indice_idx, 'pais']}

En el año 2005 :
  País con menor índice (mayor libertad): {df_america.loc[pais_min_indice_idx, 'pais']}
  País con mayor índice (menor libertad): {df_america.loc[pais_max_indice_idx, 'pais']}

En el año 2006 :
  País con me

,anio,pais
27,2001,Canadá
343,2002,Trinidad y Tobago
523,2003,Trinidad y Tobago
703,2004,Trinidad y Tobago
741,2005,Bolivia
927,2006,Canadá
1107,2007,Canadá
1287,2008,Canadá
1611,2009,Estados Unidos
1701,2012,Jamaica



Paises con mayor indice (menor libertad) en cada año:


,anio,pais
39,2001,Cuba
219,2002,Cuba
365,2003,Argentina
579,2004,Cuba
759,2005,Cuba
939,2006,Cuba
1119,2007,Cuba
1299,2008,Cuba
1479,2009,Cuba
1659,2012,Cuba


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [89]:
#Tabla

df_pivot = pd.pivot_table(df, values='indice', index='pais', columns='anio', aggfunc='max', fill_value=0)
df_pivot

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,81.3,89.17,86.88,73.25,67.25,79.25,86.17,81.67,75.75,71.78,72.36,72.63,74.27,73.96,75.05,74.93
West Bank y Gaza,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42.90,42.96,44.68
Yemen,34.8,41.83,48.00,46.25,54.00,56.67,59.00,83.38,82.13,69.22,67.26,66.36,67.07,65.80,62.23,61.66


In [90]:
#a
max_indice_pais = df_pivot.max(axis=1)
pais_max_indice = max_indice_pais.idxmax()
print(f"El país con el mayor indice es: {pais_max_indice} con indice {max_indice_pais[pais_max_indice]}")

min_indice_pais = df_pivot[df_pivot > 0].min(axis=1)
pais_min_indice = min_indice_pais.idxmin()
print(f"El país con el menor indice (distinto de cero) es: {pais_min_indice} con indice {min_indice_pais[min_indice_pais.idxmin()]}")

El país con el mayor indice es: Kosovo con indice 64536.0
El país con el menor indice (distinto de cero) es: Austria con indice 0.5


In [91]:
#b)
promedio_indice_por_anio = df_pivot.mean(axis=0)
anio_max_indice = promedio_indice_por_anio.idxmax()
anio_min_indice = promedio_indice_por_anio.idxmin()
print(f"El año con el mayor promedio de indice es:",anio_max_indice,"con promedio:",promedio_indice_por_anio[anio_max_indice])
print(f"El año con el menor promedio de indice es:",anio_min_indice, "con promedio:", promedio_indice_por_anio[anio_min_indice])

El año con el mayor promedio de indice es: 2013 con promedio: 449.11446927374294
El año con el menor promedio de indice es: 2001 con promedio: 20.03240223463687


In [92]:
#c)
variabilidad_indice_por_pais = df_pivot.max(axis=1) - df_pivot.min(axis=1)
print("Pais con mayor variabilidad en indice:",variabilidad_indice_por_pais.idxmax(), "con variabilidad:",variabilidad_indice_por_pais[variabilidad_indice_por_pais.idxmax()])

Pais con mayor variabilidad en indice: Kosovo con variabilidad: 64536.0


In [93]:
#d)
pais_constante = df_pivot[(df_pivot.max(axis=1) == df_pivot.min(axis=1))]
pais_constante

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,


No hay paises con indice constante

In [94]:
#e)

pais_sin_datos=df_pivot[df_pivot.eq(0).all(axis=1)]
pais_sin_datos

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,


No hay paises sin datos porque fueron eliminados en pasos anteriores